# **Task 3 - KPIs & Exploratory Data Analysis**

Define 3-6 Key Performance Indicators (KPIs) in addition to those used in the previous tasks and perform exploratory data analysis on multiple sheets in both Excel files. Make sure to document what you are investigating using a combination of markdown and code in the Jupyter notebook. If you are using Python scripts, write markdown alongside the code. Also, document your findings and save all visualizations.